In [1]:
from pymc_marketing.clv import utils

import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
cdnow_trans = "https://raw.githubusercontent.com/pymc-labs/pymc-marketing/main/datasets/cdnow_transactions.csv"

cdnow_df = pd.read_csv(cdnow_trans,dtype={"date":str})
#cdnow_df = cdnow_df.drop(columns={"_id","cds_bought"})

cdnow_df.head()

,_id,id,date,cds_bought,spent
0,4,1,19970101,2,29.33
1,4,1,19970118,2,29.73
2,4,1,19970802,1,14.96
3,4,1,19971212,2,26.48
4,21,2,19970101,3,63.34


`_find_first_transactions` flags the first purchase each customer has made, which must be excluded for modeling. It is called internally by `rfm_summary`

In [3]:
utils._find_first_transactions(
    transactions=cdnow_df, 
    customer_id_col = "id", 
    datetime_col = "date",
    monetary_value_col = "spent", 
    datetime_format = "%Y%m%d",
).head()

,id,date,spent,first
0,1,1997-01-01,29.33,True
1,1,1997-01-18,29.73,False
2,1,1997-08-02,14.96,False
3,1,1997-12-12,26.48,False
4,2,1997-01-01,63.34,True


`rfm_summary` is the primary data preprocessing step for CLV modeling in the continuous, non-contractual domain

In [4]:
rfm_df = utils.rfm_summary(
    cdnow_df, 
    customer_id_col = "id", 
    datetime_col = "date", 
    monetary_value_col = "spent",
    observation_period_end = '19980101',
    datetime_format = "%Y%m%d",
    time_unit = "W",
    include_first_transaction=False,
)

rfm_df.head()

,customer_id,frequency,recency,T,monetary_value
0,1,3.0,49.0,52.0,23.723333
1,2,1.0,2.0,52.0,11.770000
2,3,0.0,0.0,52.0,0.000000
3,4,0.0,0.0,52.0,0.000000
4,5,0.0,0.0,52.0,0.000000


In [5]:
print(f"Date of first purchase: {cdnow_df['date'].min()}")
print(f"Date of last purchase: {cdnow_df['date'].max()}")

rfm_df.describe()

Date of first purchase: 19970101
Date of last purchase: 19980630


,customer_id,frequency,recency,T,monetary_value
count,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000
mean,1179.000000,1.244378,10.969453,45.985999,16.719274
std,680.551615,2.406418,15.968983,3.346091,37.518144
min,1.000000,0.000000,0.000000,40.000000,0.000000
25%,590.000000,0.000000,0.000000,43.000000,0.000000
50%,1179.000000,0.000000,0.000000,46.000000,0.000000
75%,1768.000000,1.000000,22.000000,49.000000,25.850000
max,2357.000000,31.000000,52.000000,52.000000,1258.462000


For MAP fits and covariate models, `rfm_train_test_split` can be used to evaluate models on unseen data. It is also useful for identifying the impact of a time-based event like a marketing campaign.

In [6]:
utils.rfm_train_test_split(
    cdnow_df, 
    customer_id_col = "id", 
    datetime_col = "date", 
    train_period_end = "19980101",
    monetary_value_col = "spent",
    datetime_format = "%Y%m%d",
    time_unit = "W")

KeyError: "['test_T'] not in index"

In [22]:
d = [
    [1, "2015-01-01", 1],
    [1, "2015-02-06", 2],
    [2, "2015-01-01", 2],
    [3, "2015-01-01", 3],
    [3, "2015-01-02", 1],
    [3, "2015-01-05", 5],
    [4, "2015-01-16", 6],
    [4, "2015-02-02", 3],
    [4, "2015-02-05", 3],
    [5, "2015-01-16", 3],
    [5, "2015-01-17", 1],
    [5, "2015-01-18", 8],
    [6, "2015-02-02", 5],
]
test_data = pd.DataFrame(d, columns=["id", "date", "monetary_value"])

test_data["date"] = pd.to_datetime(test_data["date"],format="%Y-%m-%d")
test_transactions = test_data.loc[test_data["date"] > '2015-02-01'].copy()

test_transactions.head()

,id,date,monetary_value
1,1,2015-02-06,2
7,4,2015-02-02,3
8,4,2015-02-05,3
12,6,2015-02-02,5


The additional `_id` and `cds_bought` columns had to be dropped when data was loaded because it caused the columns assignment line to break

In [28]:
test_rfm_data = (
        test_transactions.groupby(['id', 'date'], sort=False)
        .agg(lambda r: 1)
        .groupby(level='id')
        .count()
    )#.reset_index()
# test_rfm_data["test_frequency"] = test_rfm_data.iloc[:,0]
test_rfm_data.columns= ["test_frequency"]

test_rfm_data["test_monetary_value"] = (
    test_transactions.groupby(["id", "date"])[
        "monetary_value"
    ]
    .sum()
    .groupby("id")
    .mean()
    )

test_rfm_data = test_rfm_data.reset_index().rename(
        columns={"id": "customer_id"}
    )
test_rfm_data.head()

,customer_id,test_frequency,test_monetary_value
0,1,1,2.0
1,4,2,3.0
2,6,1,5.0


In [30]:
(test_transactions["date"].max().to_period("D") - test_transactions["date"].min().to_period("D")).n

4

In [31]:
rfm_df.merge(test_rfm_data,on="customer_id", how="left")

,customer_id,frequency,recency,T,monetary_value,test_frequency,test_monetary_value
0,1,3.0,49.0,52.0,23.723333,1.0,2.0
1,2,1.0,2.0,52.0,11.770000,NaN,NaN
2,3,0.0,0.0,52.0,0.000000,NaN,NaN
3,4,0.0,0.0,52.0,0.000000,2.0,3.0
4,5,0.0,0.0,52.0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...
2352,2353,0.0,0.0,40.0,0.000000,NaN,NaN
2353,2354,5.0,24.0,40.0,44.928000,NaN,NaN
2354,2355,0.0,0.0,40.0,0.000000,NaN,NaN
2355,2356,5.0,40.0,40.0,32.450000,NaN,NaN


In [33]:
cdnow_df[cdnow_df["id"]==2355]

,_id,id,date,cds_bought,spent
6909,23554,2355,1997-03-25,1,11.77
6910,23554,2355,1998-02-01,2,24.60
